In [1]:
# Imports
import requests
import matplotlib.pyplot as plt
import numpy as np 
from pathlib import Path 
import json
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import csv
from bs4 import BeautifulSoup
from sklearn import linear_model 
from sklearn import svm
from sklearn import ensemble
import warnings 
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
import warnings 
import plotly.graph_objects as go 
import plotly.express as px 

ModuleNotFoundError: No module named 'plotly'

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
#read_csv file for "FF_qb_2020.csv" and "FF_qb_2021.csv"
qb_data_2020 = pd.read_csv("FF_qb_2020.csv")
qb_data_2021 = pd.read_csv("FF_qb_2021.csv")
qb_data_2020

In [ ]:
#Get column data for Player, Fantasy Points, and Fantasay points a game. dropna. 
qb_data_2020 = qb_data_2020[["Player" , "FPTS", "FPTS/G"]].dropna()
qb_data_2021 = qb_data_2021[["Player" , "FPTS", "FPTS/G"]].dropna()

In [ ]:
#Add years and QB's to the table names using add_prefix function
#qb_data_2020 = qb_data_2020.add_prefix("2020_QBs_")
#qb_data_2021 = qb_data_2021.add_prefix("2021_QBs_")

In [ ]:
#Display data
qb_data_2020

In [ ]:
#Get column data for Player, Fantasy Points, and Fantasay points a game. dropna. 
qb_data_2020 = qb_data_2020[["Player", "FPTS", "FPTS/G",]].dropna()
qb_data_2021 = qb_data_2021[["Player", "FPTS", "FPTS/G",]].dropna()

qb_data_2020["Year"] = 2020
qb_data_2021["Year"] = 2021

#Display
display(qb_data_2020)

#Display
display(qb_data_2021)

In [ ]:
#qb_data_2020.insert(loc=1, column="Fantasy Pos", value=["QB"])
qb_data_2020["Fantasy Pos"]= "QB"

qb_data_2021["Fantasy Pos"]= "QB"

In [ ]:
#rename qb_data_2020 and qb_data_2021 columns. 
qb_data_2020.rename(columns={
    "FPTS":"Fantasy Points",
    "FPTS/G":"Fantasy Points/GM"}, inplace=True)


qb_data_2021.rename(columns={
    "FPTS":"Fantasy Points",
    "FPTS/G":"Fantasy Points/GM"}, inplace=True)

#Display
display(qb_data_2020)
#Dispaly 
display(qb_data_2021)

In [ ]:
#move column name "Fantasy Pos" to 2nd column position.
column_name_move = qb_data_2020.pop("Fantasy Pos")

qb_data_2020.insert(1, "Fantasy Pos", column_name_move)


column_name_move = qb_data_2021.pop("Fantasy Pos")

qb_data_2021.insert(1, "Fantasy Pos", column_name_move)

#Display
display(qb_data_2020)

#Display
display(qb_data_2021)

In [ ]:
#concat qb_dataframes and dropna. 
combine_qb_data= pd.concat([qb_data_2020, qb_data_2021], axis=0, sort=False).dropna()
combine_qb_data.reset_index(inplace=True, drop=True)
combine_qb_data

In [ ]:
#create a team list and player list 
team_list = []
player_name_list = []

# create a forloop to split the player name and player team. 
for i in range(0,len(combine_qb_data["Player"])):
    team_list.append(combine_qb_data["Player"][i].split()[-1].replace("(","").replace(")",""))
    player_name_list.append(" ".join(combine_qb_data["Player"][i].split()[:-1]))

#create split list Player and Team
combine_qb_data["Player"] = player_name_list
combine_qb_data["Team"] = team_list
combine_qb_data

In [ ]:
#groupby Players get the average of fantasy points and fantasay points/GM sort value by fantasy point/gm in accending order. 
combine_qb_data.groupby(["Player"]).mean()[["Fantasy Points", "Fantasy Points/GM"]].sort_values("Fantasy Points/GM",ascending=False).reset_index()

In [ ]:
#import hvplot
import hvplot.pandas

#scatter plot the x axis fantasy points/gm and fantasy and y axis fantasy points. 
combine_qb_data.hvplot.scatter(
    y="Fantasy Points", 
    x="Fantasy Points/GM", 
    by="Year", 
    hover_cols=["Player"]
)

# Machine Learning Model Development

In [ ]:
# Review the dataframe
combine_qb_data.head()

In [ ]:
# Define target column and features columns
X = combine_qb_data.drop(columns=["Fantasy Points/GM","Year"])
y = combine_qb_data["Fantasy Points/GM"]

# Review X
X.head()

In [ ]:
# Review y
y[:5]

In [ ]:
# Encode categorial variables into numeric variables
X = pd.get_dummies(X)
X.head()

In [ ]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Review scaled data
X[:1]

In [ ]:
# Get train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [ ]:
# Fit Linear Regressin Model *bad test model and doesn't work with random_state. 
from sklearn.linear_model import LinearRegression
# Fit ExtraTreesRegressor *good test model
from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
# Model evaluation

# Get the coefficient of determination R2
from sklearn.metrics import r2_score
print(r2_score(y_test, y_pred))

In [ ]:
# Get the Mean Squared Error
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test,y_pred))

In [ ]:
# Get the Root Mean Squared Error
print(np.sqrt(mean_squared_error(y_test, y_pred)))